In [115]:
import json
import requests
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from geopy.adapters import AioHTTPAdapter
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
#%load_ext autotime
import geopandas as gpd
import geopy
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm.notebook import tqdm_notebook
import reverse_geocoder as rg


In [116]:
mcdw = pd.read_csv('../data/mcdw.20230530.csv')
mcdw.info()

C:\Users\Aaron\AppData\Local\Temp\ipykernel_38564\2137799343.py:1: DtypeWarning: Columns (9,13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  mcdw = pd.read_csv('../data/mcdw.20230530.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907855 entries, 0 to 907854
Data columns (total 36 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   WMO ID                                             907855 non-null  int64  
 1   YEAR-MONTH                                         907855 non-null  object 
 2   LONGITUDE (deg)                                    907855 non-null  float64
 3   LATITUDE (deg)                                     907855 non-null  float64
 4   ELEVATION (m)                                      907855 non-null  float64
 5   STATION PRESSURE (0.1 mb)                          907855 non-null  int64  
 6   STATION PRESSURE SOURCE FLAG                       826655 non-null  object 
 7   MEAN SEA LEVEL PRESSURE (0.1 mb)                   825674 non-null  float64
 8   MEAN SEA LEVEL PRESSURE SOURCE FLAG                907855 non-null  object

In [117]:
#Droppping columns not relavent 
mcdw.drop(['ELEVATION (m)', 'STATION PRESSURE (0.1 mb)','STATION PRESSURE SOURCE FLAG', 'MEAN SEA LEVEL PRESSURE (0.1 mb)', 'MEAN SEA LEVEL PRESSURE SOURCE FLAG', 'MEAN SEA LEVEL PRESSURE HEIGHT FLAG', 'VAPOR PRESSURE (0.1 mb)', 'VAPOR PRESSURE SOURCE FLAG', 'VAPOR PRESSURE DEPARTURE (0.1 mb)', 'VAPOR PRESSURE DEPARTURE SOURCE FLAG'], axis=1, inplace=True)

In [118]:
#removing data with unknow coordinates
#mcdw = mcdw[(mcdw['LATITUDE (deg)'] != -9999.0000) & (mcdw['LONGITUDE (deg)'] != -9999.0000)]

In [119]:
len(mcdw['LATITUDE (deg)'])

907855

In [120]:
len(mcdw['LONGITUDE (deg)'])

907855

In [121]:
#mcdw['lat_long'] = mcdw['LATITUDE (deg)'] & mcdw['LONGITUDE (deg)']
#mcdw['lat_long']=mcdw.apply(lambda x:'%s,%s' % (x['LATITUDE (deg)'],x['LONGITUDE (deg)']),axis=1)
mcdw['Coordinates'] = mcdw.apply(lambda row: (row['LATITUDE (deg)'], row['LONGITUDE (deg)']), axis=1)
#mcdw['lat_long'] = mcdw[mcdw['lat_long'].astype(str)]
#mcdw['geom'] = mcdw['LATITUDE (deg)'].map(str) + '','' + mcdw['LONGITUDE (deg)'].map(str)
mcdw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907855 entries, 0 to 907854
Data columns (total 27 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   WMO ID                                             907855 non-null  int64  
 1   YEAR-MONTH                                         907855 non-null  object 
 2   LONGITUDE (deg)                                    907855 non-null  float64
 3   LATITUDE (deg)                                     907855 non-null  float64
 4   MINIMUM TEMPERATURE (deg C)                        907855 non-null  int64  
 5   MINIMUM TEMPERATURE SOURCE FLAG                    605950 non-null  object 
 6   MEAN TEMPERATURE (deg C)                           907855 non-null  int64  
 7   MEAN TEMPERATURE SOURCE FLAG                       892136 non-null  object 
 8   MEAN TEMPERATURE DEPARTURE (deg C)                 907855 non-null  int64 

In [122]:
mcdw['lat_long'] = mcdw['lat_long'].astype(str)

KeyError: 'lat_long'

In [ ]:
#other option of creating tuple list for the geolocator
lat = mcdw['LATITUDE (deg)']
lon = mcdw['LONGITUDE (deg)']

result = []
for i in range(len(lat)):
     #Adding to result based on indexes
     result.append((lat[i], lon[i]))

In [ ]:
result

[(-9999.0, -9999.0),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8.6678),
 (70.9397, -8

In [ ]:
#Need to group into regions
#lat = mcdw['LATITUDE (deg)']
#lang = mcdw['LONGITUDE (deg)']
#location = geolocator.reverse("{}, {}".format(mcdw['LATITUDE (deg)'], mcdw['LONGITUDE (deg)']))

geolocator = Nominatim(user_agent="google")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#location = geolocator.reverse(mcdw['lat_long'])
 #df['location'] = df['name'].apply(geocode)
#mcdw["location"] = mcdw['lat_long'].apply(lambda el: geolocator.reverse(el).raw["address"])
#mcdw["city"] = list(map(lambda el: geolocator.reverse(el).raw["address"], mcdw["lat_long"]))
mcdw["city"] = list(map(lambda el: geolocator.reverse(el).raw["address"], result))

#mcdw['location'] = geolocator.reverse(f'{lat}, {lang}')

ValueError: Must be a coordinate pair or Point

In [ ]:
locator = Nominatim(user_agent='google', timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

In [ ]:
#type(result[0])
location = locator.reverse(result[0])
#location.raw

c:\Users\Aaron\anaconda3\lib\site-packages\geopy\point.py:472: UserWarning: Latitude normalization has been prohibited in the newer versions of geopy, because the normalized value happened to be on a different pole, which is probably not what was meant. If you pass coordinates as positional args, please make sure that the order is (latitude, longitude) or (y, x) in Cartesian terms.
  return cls(*args)


ValueError: Must be a coordinate pair or Point

In [127]:
#So that made a nice little dic of locations... 

location_dic = rg.search(result)